In [3]:
import pandas as pd

data = pd.read_csv("./StormEvents_details-ftp_v1.0_d1996_c20170717.csv")

/Users/mdevlin/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
data.columns

Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE'],
      dtype='object')

In [13]:
data.columns = [x.lower() for x in list(data.columns)]

In [14]:
data.columns

Index(['begin_yearmonth', 'begin_day', 'begin_time', 'end_yearmonth',
       'end_day', 'end_time', 'episode_id', 'event_id', 'state', 'state_fips',
       'year', 'month_name', 'event_type', 'cz_type', 'cz_fips', 'cz_name',
       'wfo', 'begin_date_time', 'cz_timezone', 'end_date_time',
       'injuries_direct', 'injuries_indirect', 'deaths_direct',
       'deaths_indirect', 'damage_property', 'damage_crops', 'source',
       'magnitude', 'magnitude_type', 'flood_cause', 'category', 'tor_f_scale',
       'tor_length', 'tor_width', 'tor_other_wfo', 'tor_other_cz_state',
       'tor_other_cz_fips', 'tor_other_cz_name', 'begin_range',
       'begin_azimuth', 'begin_location', 'end_range', 'end_azimuth',
       'end_location', 'begin_lat', 'begin_lon', 'end_lat', 'end_lon',
       'episode_narrative', 'event_narrative', 'data_source'],
      dtype='object')

In [42]:
data[["episode_narrative"]].loc[0, "episode_narrative"]

'Strong gradient winds followed behind a line of thunderstorms which occurred along a strong cold front.  This caused widespread damage to trees and power lines.  Numerous tin sheds and a few barns were destroyed during the 15 to 20 hours of strong to severe winds.  Power outages were too numerous to count and one of these outages was credited with starting a furnace fire.  The fire however was confined and no dollar damage was reported.  In spite of the widespread damage to trees, several newspaper articles across the area reported that damage to unharvested crops was minimal.'

In [35]:
data.groupby("event_type").mean()[["deaths_direct"]]

,deaths_direct
event_type,
Avalanche,0.714286
Blizzard,0.015374
Coastal Flood,0.019900
Cold/Wind Chill,0.024050
Debris Flow,0.041667
Dense Fog,0.007353
Drought,0.000000
Dust Devil,0.090909
Dust Storm,0.000000


In [44]:
data.shape

(48561, 51)

In [49]:
data[data["damage_property"].notnull()]["damage_property"]

5         25K
6        .02M
7         10K
13        20K
14        20K
23        .1M
25          0
28        20K
29        20K
46        10K
52        20K
63         1K
68         5K
69       7.9M
70        .1M
129      .02M
132      .04M
171       15K
173       20K
209      .06M
222       20K
223       90K
226        1K
237       30K
242       15K
245      .05M
249       .5K
250       10K
257        4K
268        1K
         ... 
48167      4K
48168      4K
48171      5K
48174      2K
48175    1.5K
48176     15K
48179     50K
48239      1K
48253     20K
48254     10K
48258     20K
48272      5K
48283      5K
48343      5K
48362     40K
48365     80K
48373      3K
48430     30K
48435    .06M
48439     .3M
48456      5K
48482     10K
48484     10K
48511       0
48520     10K
48528     15M
48533     25M
48540     .1M
48543     15K
48556     25K
Name: damage_property, Length: 11587, dtype: object